In [2]:
#!pip install elevation

In [1]:
# import elevation

# # Set the bounds (example: Denver area)
# #bounds = (-105.5, 39.5, -104.5, 40.5)
# raster = rasterio.open()
# output = "/Users/cmbreen/code/gap-filling/data_processing/elevation_download/test_output_dem.tif"

# # Download and clip the SRTMGL1 (30m) data
# # Note: Requires an active internet connection for initial download, then caches locally
# # DEM product choice.  [default: SRTM1] but could also do STRM3
# elevation.clip(bounds=bounds, output=output)


In [2]:
# def elevation_download(raster):

In [3]:
"""
elevation_download.py

Script to download and clip SRTM elevation data for a given raster file or bounding box.
Uses the elevation package to download SRTM1 (30m) or SRTM3 (90m) data.
"""

import rasterio
import xarray as xr
import shapely
import elevation
import os
from pathlib import Path
import glob
from rasterio.warp import transform_bounds

In [4]:

##### single file example ###
os.makedirs('outputs', exist_ok=True) 

input_raster_path = '/Volumes/MyBook/aso_data/swe_tifs/colorado/Uncompahgre River/ASO_50M_SWE_USCOUB_20140320.tif'
filename = input_raster_path.split('/')[-1]
output_dem_path = f'dem_{filename}'
print(output_dem_path)

# Open the input raster and get bounds
with rasterio.open(input_raster_path) as src:
    bounds = src.bounds  
    crs = src.crs
    print(crs)
    print(f"Extracted bounds from {input_raster_path}:")
    print(f"  Bounds: {bounds}")
    bounds_4326 = transform_bounds(str(crs), "EPSG:4326", *bounds)

print(bounds_4326)
#bounds_4326[0] bounds_4326[1] bounds_4326[3] bounds_4326[4]
#os.system(f"eio clip -o dem_{filename} --bounds {str(bounds_4326[0])} {str(bounds_4326[1])} {str(bounds_4326[2])} {str(bounds_4326[3])}")


dem_ASO_50M_SWE_USCOUB_20140320.tif
EPSG:32613
Extracted bounds from /Volumes/MyBook/aso_data/swe_tifs/colorado/Uncompahgre River/ASO_50M_SWE_USCOUB_20140320.tif:
  Bounds: BoundingBox(left=239712.0, bottom=4197092.0, right=274712.0, top=4231992.0)
(-107.97235107690884, 37.88415338070932, -107.56208732361442, 38.20771599706473)


In [5]:
### function for all files ### 
## doing it on local system because we don't have elevation on discover, but we could put it there pretty easily ##
os.makedirs('outputs', exist_ok=True)  # Add this line

input_raster_paths = glob.glob('/Volumes/MyBook/aso_data/swe_tifs/colorado/**/*.tif')

buffer_degrees = 0.1  # You can adjust this (1-2 degrees as needed)

for input_raster_path in input_raster_paths:
    #if i > 1: break
    filename = input_raster_path.split('/')[-1]
    output_dem_path = f'dem_{filename}'
    # Open the input raster and get bounds
    with rasterio.open(input_raster_path) as src:
        bounds = src.bounds  
        crs = src.crs
        print(crs)
        print(f"Extracted bounds from {input_raster_path}:")
        print(f"  Bounds: {bounds}")
        bounds_4326 = transform_bounds(str(crs), "EPSG:4326", *bounds)
        buffered_bounds = [
        bounds_4326[0] - buffer_degrees,  # west (minx) - subtract
        bounds_4326[1] - buffer_degrees,  # south (miny) - subtract
        bounds_4326[2] + buffer_degrees,  # east (maxx) - add
        bounds_4326[3] + buffer_degrees   # north (maxy) - add
        ]
    
    print(buffered_bounds)
    #bounds_4326[0] bounds_4326[1] bounds_4326[3] bounds_4326[4]
    os.system(f"eio clip -o outputs/dem_{filename} --bounds {str(buffered_bounds[0])} {str(buffered_bounds[1])} {str(buffered_bounds[2])} {str(buffered_bounds[3])}")


EPSG:32613
Extracted bounds from /Volumes/MyBook/aso_data/swe_tifs/colorado/Uncompahgre River/ASO_50M_SWE_USCOUB_20140320.tif:
  Bounds: BoundingBox(left=239712.0, bottom=4197092.0, right=274712.0, top=4231992.0)
[-108.07235107690883, 37.78415338070932, -107.46208732361443, 38.30771599706473]
make: Nothing to be done for `download'.
make: Nothing to be done for `all'.
cp SRTM1.vrt SRTM1.8f2ba25e46cd4e2cbf61eb1d8a47fe0e.vrt
gdal_translate -q -co TILED=YES -co COMPRESS=DEFLATE -co ZLEVEL=9 -co PREDICTOR=2 -projwin -108.07235107690883 38.30771599706473 -107.46208732361443 37.78415338070932 SRTM1.8f2ba25e46cd4e2cbf61eb1d8a47fe0e.vrt /Users/cmbreen/code/gap-filling/data_processing/elevation_download/outputs/dem_ASO_50M_SWE_USCOUB_20140320.tif
rm -f SRTM1.8f2ba25e46cd4e2cbf61eb1d8a47fe0e.vrt
EPSG:32613
Extracted bounds from /Volumes/MyBook/aso_data/swe_tifs/colorado/Animas/ASO_Animas_Mosaic_2021Apr19_swe_50m.tif:
  Bounds: BoundingBox(left=231453.0, bottom=4129448.847759966, right=278603.095

In [8]:
buffered_bounds

[-108.7082581079061, 38.46865783925005, -105.08100770666931, 42.41177236701919]

In [ ]:
## now rsync
#scp -r /Users/cmbreen/code/gap-filling/data_processing/elevation_download/outputs_new cmbreen@discover.nccs.nasa.gov:/discover/nobackup/cmbreen/gap-filling-data 


In [9]:
bounds_4326

(-107.2082581079061, 39.96865783925005, -106.58100770666931, 40.91177236701919)

In [ ]:
## onto discover: 
#scp -r /Users/cmbreen/code/gap-filling/data_processing/landcover_download/Annual_NLCD_LndCov_2019_CU_C1V1/Annual_NLCD_LndCov_2019_CU_C1V1.tif cmbreen@discover.nccs.nasa.gov:/discover/nobackup/cmbreen/gap-filling-data 


#scp -r cmbreen@discover.nccs.nasa.gov:/discover/nobackup/cmbreen/gap-filling-data/elevation_30m/dem_ASO_50M_SWE_USCOUB_20140320.tif /Users/cmbreen/Documents/snow/gap_filling

#scp -r cmbreen@discover.nccs.nasa.gov:/discover/nobackup/cmbreen/gap-filling-data/elevation_50m/dem_ASO_YampaRiver_2024Apr11_swe_50m.tif /Users/cmbreen/Documents/snow/gap_filling


'/Volumes/MyBook/aso_data/swe_tifs/colorado/Uncompahgre River/ASO_50M_SWE_USCOUB_20140320.tif'

scp -r /Volumes/MyBook/aso_data/swe_tifs/colorado/Uncompahgre River/ASO_50M_SWE_USCOUB_20140320.tif cmbreen@discover.nccs.nasa.gov:/discover/nobackup/cmbreen/aso_data/swe_tifs/colorado 

